# Экономика преступления и наказания

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import urllib.request
import os
from os.path import join

In [2]:
%pylab inline
from matplotlib import pyplot as plt

Populating the interactive namespace from numpy and matplotlib


### Baseline model

Для начала соберем данные о преступности, безработице и среднем заработке населения.
Я нашла два источника данных о преступности (количество преступлений на 100 тыс. человек) за разные периоды и объединила их в одну таблицу. К сожалению, доступ на многие страницы на сайте www.gks.ru, где собраны статистические выборки, ограничен. Поэтому я ограничилась тем, что оказалось доступно.

In [3]:
url='https://realnoevremya.ru/attachments/1015'

page=requests.get(url)

soup=BeautifulSoup(page.text, 'html.parser')

In [6]:
array=np.array([x.text.replace('\n','').replace('\t','').replace('\r','').replace('.','').replace(',','.') for x in soup.findAll('table')[0].findAll('td')]).reshape(-1,7)
df=pd.DataFrame({int(array[0,i]):array[1:,i].astype(float) for i in range(1,len(array[0])-1)}, index=array[1:,0])

Первая таблица - за 2014 - 2018 годы

In [216]:
df.head()

,2018,2017,2016,2015,2014
Республика Тыва,2876.4,3662.8,3457.8,2689.5,2113.2
Республика Бурятия,2388.1,2399.1,2390.2,2840.8,2525.2
Забайкальский край,2268.9,2501.6,2738.5,3068.9,2948.6
Еврейская автономная область,2260.9,2199.5,2061.2,2152.4,2033.7
Амурская область,2196.3,2308.1,2315.5,2404.7,2126.8


In [9]:
url1='https://www.gks.ru/bgd/regl/b12_14p/isswww.exe/stg/d01/09-01.htm'
page1=requests.get(url1)
page1.encoding='windows-1251'
soup1 = BeautifulSoup(page1.text, 'lxml')

In [10]:
soup1 = BeautifulSoup(page1.text, 'lxml')
full_list=[x.text.replace('\n','').replace('\r',' ') for x in soup1.table.find_all('td')][:-1]

In [11]:
array = np.array(full_list).reshape(-1,15)
regions = array[1:,0]
numbers = np.copy(array[1:,1:])
for i,row in enumerate(array[1:,1:]):
    for j,number in enumerate(row):
        try:
            numbers[i,j]=float(number)
        except Exception as e:
            numbers[i,j]=np.nan

In [12]:
df1=pd.DataFrame({int(array[0,i]):numbers[:,i-1].astype(float) for i in range(1,len(array[0])-1)}, index=regions)

И вторая - за 1990 и 2000-2011 годы.

In [215]:
df1.head()

,1990,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011
Российская Федерация,1243.0,2014.0,2033.0,1739.0,1906.0,2009.0,2477.0,2695.0,2509.0,2249.0,2097.0,1840.0,1682.0
Центральный федеральный округ,882.0,1562.0,1580.0,1496.0,1694.0,1734.0,2048.0,2119.0,2001.0,1811.0,1747.0,1566.0,1465.0
Белгородская область,882.0,1323.0,1350.0,1198.0,1264.0,1280.0,1670.0,1594.0,1396.0,1256.0,1307.0,1099.0,1036.0
Брянская область,1067.0,2107.0,1981.0,1510.0,1809.0,1856.0,2321.0,2191.0,2148.0,2049.0,1982.0,1833.0,1533.0
Владимирская область,954.0,2031.0,2131.0,1749.0,1913.0,2140.0,2471.0,2497.0,2296.0,1956.0,1898.0,1775.0,1659.0


Таблица crime объединяет данные за оба периода.

In [14]:
crime = df1.join(df)

Теперь найдем данные о безработице и среднем заработке.

In [16]:
unemployment_url='https://www.gks.ru/storage/mediabank/trud6_15-72.xls'

In [17]:
urllib.request.urlretrieve(unemployment_url, join(os.getcwd(),'unemployment.xls'))


('/home/alya/Downloads/unemployment.xls',
 <http.client.HTTPMessage at 0x7ff0c512ad30>)

In [18]:
unemp = pd.read_excel(join(os.getcwd(),'unemployment.xls'), sheet_name='Год 15-72 лет ', header=3, index_col=0)

Пропущенные значения в таблице обозначены многоточием, уберем его.

In [219]:
unemp=unemp.replace('…', np.nan)
unemp.head()

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Российская Федерация,10.6,9.0,7.9,8.2,7.8,7.1,7.1,6.0,6.2,8.3,7.3,6.5,5.5,5.5,5.2,5.6,5.5,5.2,4.8,4.6
Центральный федеральный округ,7.8,6.0,5.1,5.1,4.7,4.3,4.0,3.1,3.6,5.8,4.6,4.1,3.1,3.3,3.1,3.5,3.5,3.2,2.9,2.9
Белгородская область,6.1,6.4,8.1,8.3,6.2,5.9,5.6,4.3,3.9,4.7,5.2,4.3,3.7,4.0,4.0,4.1,4.0,3.9,4.0,3.9
Брянская область,13.4,9.8,8.6,7.4,8.9,6.8,6.8,6.4,6.5,10.7,8.0,7.1,5.1,5.2,5.0,4.6,4.6,4.4,3.9,3.8
Владимирская область,12.0,10.2,10.4,10.2,9.1,9.0,10.8,6.7,5.7,8.7,6.1,5.7,4.4,3.8,4.3,5.6,5.6,4.8,4.7,4.0


In [62]:
salary_url='https://www.gks.ru/storage/mediabank/t4.xlsx'
urllib.request.urlretrieve(salary_url, join(os.getcwd(),'salary.xls'))
salary_df_0_17 = pd.read_excel(join(os.getcwd(),'salary.xls'), sheet_name='2000-2017', header=3, index_col=0)
salary_df_18 = pd.read_excel(join(os.getcwd(),'salary.xls'), sheet_name='с 2018', header=3, index_col=0, 
                              usecols=[0,1,2], names=['Region',2018,2019],
                            na_values='…')
salary = salary_df_0_17.join(salary_df_18)

Можно заметить, что в таблице с зарплатами Москва и Санкт-Петербург написаны без пробела, так что исправим это, чтобы унифицировать все данные.

In [67]:
as_list = salary.index.tolist()
idx = as_list.index('г.Санкт-Петербург')
as_list[idx] = 'г. Санкт-Петербург'
salary.index = as_list


as_list = unemp.index.tolist()
idx = as_list.index('г.Санкт-Петербург')
as_list[idx] = 'г. Санкт-Петербург'
unemp.index = as_list

In [68]:
salary=salary.replace('…', np.nan)

Теперь переиндексируем все таблицы, приведя их к виду (Регион+Год, значение).

In [146]:
salary_df=salary.unstack().to_frame().sort_index(level=1).rename(columns={0:'salary'})
unemp_df=unemp.unstack().to_frame().sort_index(level=1).rename(columns={0:'unemployment'})
crime_df=crime.unstack().to_frame().sort_index(level=1).rename(columns={0:'criminality'})


In [220]:
crime_df.dropna(subset = ["criminality"], inplace=True)
crime_df.head()

,,criminality
1990,Алтайский край,1216.0
2000,Алтайский край,2143.0
2001,Алтайский край,2093.0
2002,Алтайский край,2057.0
2003,Алтайский край,2434.0


Объединим все данные в одну таблицу

In [303]:
full_table=crime_df.join(salary_df.join(unemp_df))

In [304]:
full_table=full_table.replace('…', np.nan)

In [234]:
full_table.head()

,,criminality,salary,unemployment
2000,Алтайский край,2143.0,1364.8,11.7
2001,Алтайский край,2093.0,1870.4,10.0
2002,Алтайский край,2057.0,2567.6,8.3
2003,Алтайский край,2434.0,3147.5,12.3
2004,Алтайский край,2315.0,3955.3,9.1


Создадим матрицу признаков объектов и вектор меток.

In [235]:
X = np.zeros((len(full_table),2))
X[:,0] = (np.array(full_table['salary']))
X[:,1] = (np.array(full_table['unemployment']))
y  = (np.array(full_table['criminality']))

In [236]:
import numpy as np
from sklearn.model_selection import train_test_split

Разделим на обучающую и тестовую выборки.

In [238]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, shuffle=True,random_state=42)

Для начала попробуем самую простую линейную регрессию. В каждой из последующих моделей я вывожу r2 скор на обучающей и тестовой выборках.

In [240]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)
print(reg.score(X_train, y_train))
print(reg.score(X_test, y_test))

0.08809819946190478
-0.016514627991753184


Скор на тестовой выборке получился отрицательный, то есть такая модель вообще не обучилась и не предсказывает метки. Попробуем использовать метод ближайшего соседа. Подребем колчество ближайших соседей и метрику близости признаков.

In [245]:
from sklearn.model_selection import GridSearchCV
scoring = {'r2': 'r2'}
gs = GridSearchCV(KNeighborsRegressor(),
                  param_grid={'n_neighbors': range(2, 50, 1), 'weights':['uniform', 'distance'], 'p':[1,2] },
                  scoring=scoring, refit='r2', return_train_score=True, n_jobs=-1)
gs.fit(X_train, y_train)
gs.best_params_

{'n_neighbors': 48, 'p': 1, 'weights': 'uniform'}

In [246]:
neigh = KNeighborsRegressor(n_neighbors=48,p=1)
neigh.fit(X_train, y_train)
print(neigh.score(X_train, y_train))
print(neigh.score(X_test, y_test))

0.1517006319094949
0.05919827720921789


Уже положительно, но не далеко ушло от нуля.
Попробуем деревья решений: случайный лес и градиентный бустинг.

In [247]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=1000,max_depth=3, random_state=0)
rf.fit(X_train, y_train)
print(rf.score(X_train, y_train))
print(rf.score(X_test, y_test))

0.29213141596360914
0.17980887522235334


In [248]:
from sklearn.ensemble import GradientBoostingRegressor as GBR
gbr = GBR(n_estimators=50, max_depth=2)
gbr.fit(X_train, y_train)
print(gbr.score(X_train, y_train))
print(gbr.score(X_test, y_test))

0.34985598509660343
0.19793526220610524


Градиентный бустинг показывает $R^2$ уже почти 20%. Не ноль, но предсказания пока очень далеки от идеала.

Чтобы понять, работает ли теория Беккера, оценим вероятность поимки преступника. На сайте, где я нашла часть данных о преступности, была также информация о раскрываемости преступлений.
Поэтому предлагаю оценить вероятность поимки как отношения количества раскрытых преступлений к общему числу преступлений.

In [249]:
url='https://realnoevremya.ru/attachments/1016'
page_crime = requests.get(url)
soup=BeautifulSoup(page_crime.text, 'html.parser')

array=np.array([x.text.replace('\n','').replace('\t','').replace('\r','').replace('.','').replace(',','.') for x in soup.findAll('table')[0].findAll('td')]).reshape(-1,7)[:,:-1]
regions = array[1:,0]
numbers = np.copy(array[1:,1:])
for i,row in enumerate(array[1:,1:]):
    for j,number in enumerate(row):
        try:
            numbers[i,j]=float(number)
        except Exception as e:
            numbers[i,j]=np.nan


crime_count_df=pd.DataFrame({int(array[0,i]):numbers[:,i-1].astype(float) for i in range(1,len(array[0])-1)}, index=regions)
crime_count_df=crime_count_df.unstack().to_frame().sort_index(level=1).rename(columns={0:'crime_count'})

In [250]:
url='https://realnoevremya.ru/attachments/1018'
page_crime = requests.get(url)
soup=BeautifulSoup(page_crime.text, 'html.parser')

array=np.array([x.text.replace('\n','').replace('\t','').replace('\r','').replace('.','').replace(',','.') for x in soup.findAll('table')[0].findAll('td')]).reshape(-1,7)
crime_solved_df=pd.DataFrame({int(array[0,i]):array[1:,i].astype(float) for i in range(1,len(array[0])-1)}, index=array[1:,0])
crime_solved_df=crime_solved_df.unstack().to_frame().sort_index(level=1).rename(columns={0:'crime_solved'})

In [251]:
crimes = crime_solved_df.join(crime_count_df).dropna()
probabilities = crimes['crime_solved']/crimes['crime_count']

In [252]:
crimes['probability'] = crime_solved_df['crime_solved']/crime_count_df['crime_count']

Таблица crimes содержит информацию об общем числе преступлений и раскрываемости.

In [255]:
crimes.head()

,,crime_solved,crime_count,probability
2015,Алтайский край,26665.0,48582.0,0.548866
2016,Алтайский край,26039.0,44576.0,0.584148
2017,Алтайский край,23465.0,40055.0,0.585819
2018,Алтайский край,22786.0,38413.0,0.593185
2015,Амурская область,8917.0,19475.0,0.457869


Величины $U(W_L)$ оценим как средний заработок. Остается $U(W_S)$ и $S$. 
Так как мы оцениваем очень обобщенную модель, без привязки к конкретным преступлениям, то предположим, что потери приступника при поимке пропорциональны зарплате (вероятно, пойманный преступник лишается работы в течение некоторого времени не получает зарплату). Пусть штраф составляет полугодовую заработную плату. Тогда можно оценить, сколько должно приносить преступление в работающей теории Беккера:
$$(1-p)U(W_S)-pS>U(W_L)$$
$$(1-p)*mean(U(W_S))-p*mean(S)>mean(U(W_L))$$
$$mean(U(W_S))>\frac{p*mean(S)+mean(U(W_L))}{1-p}$$

In [256]:
Bekker = crimes.join(salary_df).dropna()

In [268]:
Bekker['crime_cost']=((Bekker['salary']+Bekker['probability']*Bekker['salary']*6)/(1-Bekker['probability']))

In [269]:
np.mean(Bekker['crime_cost'])

275547.31464755005

То есть, чтобы человеку в среднем было выгоднее совершить преступление, чем не совершать, выгода от преступления должна быть порядка 275 тыс.р. 

In [270]:
Bekker.sort_values(by='crime_cost')

,,crime_solved,crime_count,probability,salary,crime_cost
2015,Воронежская область,12701.0,39298.0,0.323197,24906,108160
2017,Ивановская область,5675.0,15943.0,0.355956,23470,114271
2015,Псковская область,4623.0,10944.0,0.422423,21553,131896
2016,Ивановская область,5984.0,14405.0,0.415411,22144,132293
2017,Калининградская область,5555.0,17231.0,0.322384,30580,132422
...,...,...,...,...,...,...
2018,Чеченская Республика,2673.0,3330.0,0.802703,26177,771684
2017,Республика Саха (Якутия),7956.0,12399.0,0.641665,62206,841944
2018,Республика Саха (Якутия),7586.0,11936.0,0.635556,68871,909604
2016,Магаданская область,1904.0,2986.0,0.637642,69769,929179


Самые "дешевые" преступления были в Воронежской области в 2015, а самые "дорогие" -- в Магаданской области в 2017, выгода должна быть почти миллион.

### User model

Теперь попробуем улучшить модель. Для начала обойдемся теми же признаками: безработицей и зарплатой, но преобразуем их, посчитав обратную величину, корень, лорафим и их произведение

In [271]:
functions=[]
functions.append(lambda x: 1/x)
functions.append(lambda x: x**.5)
functions.append(lambda x: np.log(x))
# functions.append(lambda x: np.exp(x)/np.sum(np.exp(x)))

In [272]:
X = X[:,:2]
for i in range(2):
    for fn in functions:
        X =np.concatenate((X,fn(X[:,i][:,None])), axis=1)
        print(i, X.shape)

0 (1197, 3)
0 (1197, 4)
0 (1197, 5)
1 (1197, 6)
1 (1197, 7)
1 (1197, 8)


In [273]:
X=np.concatenate((X,(X[:,0]*X[:,1])[:,None]), axis=1)

In [274]:
X_train, X_test, y_train, y_test = train_test_split(
    X[:,2:], y, test_size=0.25, shuffle=True,random_state=42)

Линейная регрессия уже начала чуть-чуть справляться.

In [276]:
reg = LinearRegression().fit(X_train, y_train)
print(reg.score(X_train, y_train))
print(reg.score(X_test, y_test))

0.18545571462481314
0.13286680522068373


Подберем параметры для случайного леса и градиентного бустинга.

In [280]:
scoring = {'r2': 'r2'}
gs = GridSearchCV(RandomForestRegressor(),
                  param_grid={'n_estimators': range(100, 401, 100), 'max_depth':[1,2,3],  },
                  scoring=scoring, refit='r2', return_train_score=True, n_jobs=-1)
gs.fit(X_train, y_train)
gs.best_params_

{'max_depth': 3, 'n_estimators': 100}

In [281]:
rf = RandomForestRegressor(n_estimators=400,max_depth=3, random_state=0)
rf.fit(X_train, y_train)
print(rf.score(X_train, y_train))
print(rf.score(X_test, y_test))

0.3057475815272708
0.20534732215548102


In [282]:
gs = GridSearchCV(GBR(),
                  param_grid={'n_estimators': range(100, 1001, 100), 'max_depth':[1,2,3], 'max_features':[1,2] },
                  scoring=scoring, refit='r2', return_train_score=True,n_jobs=-1)
gs.fit(X_train, y_train)
gs.best_params_

{'max_depth': 1, 'max_features': 2, 'n_estimators': 200}

In [283]:
gbr = GBR(n_estimators=400, max_depth=1, max_features=2)
gbr.fit(X_train, y_train)
print(gbr.score(X_train, y_train))
print(gbr.score(X_test, y_test))

0.3588246548606987
0.21479881412612722


Качество моделей на деревьях улучшилось совсем немного.
Добавим еще средние признаки безработицы по годам и регионам.

In [157]:
year_mean_unemp = unemp.agg(['mean'])
year_mean_unemp = pd.DataFrame({col_name:len(unemp)*list(year_mean_unemp[col_name]) for col_name in unemp.columns},
                              index=unemp.index)
year_mean_unemp = year_mean_unemp.unstack().to_frame().sort_index(level=1).rename(columns={0:'year_mean_unemp'})

region_mean_unemp = unemp.agg(['mean'], axis=1)
region_mean_unemp = pd.DataFrame({idx:region_mean_unemp['mean'] for idx in unemp.columns}, index=unemp.index)
region_mean_unemp = region_mean_unemp.unstack().to_frame().sort_index(level=1).rename(columns={0:'region_mean_unemp'})

In [305]:
full_table = (full_table.join(year_mean_unemp).
                        join(region_mean_unemp))

In [306]:
full_table.head()

,,criminality,salary,unemployment,year_mean_unemp,region_mean_unemp
1990,Алтайский край,1216.0,NaN,NaN,NaN,NaN
2000,Алтайский край,2143.0,1364.8,11.7,12.098925,8.525
2001,Алтайский край,2093.0,1870.4,10.0,10.481720,8.525
2002,Алтайский край,2057.0,2567.6,8.3,9.337634,8.525
2003,Алтайский край,2434.0,3147.5,12.3,9.872043,8.525


Кроме того, добавим данные об абсолютном количестве безработных и занятости (это не то же самое, что безработица, т.к. безработица не учитывает людей, не входящих в рабочую силу, а занятость - учитывает). 
И добавим данные о доходах (они отличаются от зарплат, т.к. доходы могут быть не от работы, а, например, от сдачи жилья)

In [288]:
employment_url='https://www.gks.ru/storage/mediabank/trud5_15-72.xls'
urllib.request.urlretrieve(employment_url, join(os.getcwd(),'employment.xls'))
emp = pd.read_excel(join(os.getcwd(),'employment.xls'), sheet_name='Год 15-72 лет ', header=3, index_col=0)
emp=emp.replace('…', np.nan)
emp_df=emp.unstack().to_frame().sort_index(level=1).rename(columns={0:'employment'})

In [289]:
unemployment_count_url='https://www.gks.ru/storage/mediabank/trud3_15-72.xls'
urllib.request.urlretrieve(unemployment_count_url, join(os.getcwd(),'unemployment_count.xls'))
unemp_count = pd.read_excel(join(os.getcwd(),'unemployment_count.xls'), sheet_name='Год 15-72 лет ', header=3, index_col=0)
unemp_count=unemp_count.replace('…', np.nan)
unemp_count_df=unemp_count.unstack().to_frame().sort_index(level=1).rename(columns={0:'unemployment_count'})

In [290]:
income_url='http://www.gks.ru/free_doc/new_site/population/urov/2009-2012kv.xls'
urllib.request.urlretrieve(unemployment_count_url, join(os.getcwd(),'2009-2012kv.xls'))

income = pd.read_excel(join(os.getcwd(),'2009-2012kv.xls'), sheet_name='Год 15-72 лет ', header=3, index_col=0)
income=income.replace('…', np.nan)
income_df=income.unstack().to_frame().sort_index(level=1).rename(columns={0:'income'})

И средние этих показателей по годам/регионам.

In [291]:
year_mean_income = income.agg(['mean'])
year_mean_income = pd.DataFrame({col_name:len(income)*list(year_mean_income[col_name]) for col_name in income.columns},
                              index=income.index)
year_mean_income = year_mean_income.unstack().to_frame().sort_index(level=1).rename(columns={0:'year_mean_income'})

region_mean_income = income.agg(['mean'], axis=1)
region_mean_income = pd.DataFrame({idx:region_mean_income['mean'] for idx in income.columns}, index=income.index)
region_mean_income = region_mean_income.unstack().to_frame().sort_index(level=1).rename(columns={0:'region_mean_income'})

In [292]:
year_mean_emp = emp.agg(['mean'])
year_mean_emp = pd.DataFrame({col_name:len(emp)*list(year_mean_emp[col_name]) for col_name in emp.columns},
                              index=emp.index)
year_mean_emp = year_mean_emp.unstack().to_frame().sort_index(level=1).rename(columns={0:'year_mean_emp'})

region_mean_emp = emp.agg(['mean'], axis=1)
region_mean_emp = pd.DataFrame({idx:region_mean_emp['mean'] for idx in emp.columns}, index=emp.index)
region_mean_emp = region_mean_emp.unstack().to_frame().sort_index(level=1).rename(columns={0:'region_mean_emp'})

In [307]:
full_table = (full_table.join(income_df)
                        .join(emp_df)
                        .join(unemp_count_df)
                        .join(year_mean_income)
                        .join(region_mean_income)
                        .join(year_mean_emp)
                        .join(region_mean_emp)
             ).dropna()

Вот как теперь выглядит таблица признаков.

In [308]:
full_table.head()

,,criminality,salary,unemployment,year_mean_unemp,region_mean_unemp,income,employment,unemployment_count,year_mean_income,region_mean_income,year_mean_emp,region_mean_emp
2000,Алтайский край,2143.0,1364.8,11.7,12.098925,8.525,148.973,55.4,148.973,251.308753,104.7034,57.713978,58.98
2001,Алтайский край,2093.0,1870.4,10.0,10.481720,8.525,123.691,55.0,123.691,209.820022,104.7034,57.694624,58.98
2002,Алтайский край,2057.0,2567.6,8.3,9.337634,8.525,106.076,58.1,106.076,186.075376,104.7034,58.972043,58.98
2003,Алтайский край,2434.0,3147.5,12.3,9.872043,8.525,155.305,54.9,155.305,193.757344,104.7034,58.497849,58.98
2004,Алтайский край,2315.0,3955.3,9.1,9.347312,8.525,117.596,59.0,117.596,184.970333,104.7034,59.308602,58.98


Построим по этой таблице матрицу признаков, метки, разделим на трейн/тест и обучим модели.

In [309]:
X = np.zeros((full_table.shape[0], full_table.shape[1]-1))
for i, col_name in enumerate(full_table.columns[1:]):
    print(col_name)
    X[:,i] = full_table[col_name]
y  = (np.array(full_table['criminality']))

salary
unemployment
year_mean_unemp
region_mean_unemp
income
employment
unemployment_count
year_mean_income
region_mean_income
year_mean_emp
region_mean_emp


In [310]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, shuffle=True,random_state=42)

In [311]:
gs = GridSearchCV(RandomForestRegressor(),
                  param_grid={'n_estimators': range(100, 200, 100), 
                              'max_depth':range(1,21,2), 
                              'max_features':range(1,13,2)},
                  scoring=scoring, refit='r2', return_train_score=True, n_jobs=-1)
gs.fit(X_train, y_train)
gs.best_params_

{'max_depth': 13, 'max_features': 7, 'n_estimators': 100}

In [313]:
rf = RandomForestRegressor(n_estimators=100, max_depth=13, max_features=7, random_state=0)
rf.fit(X_train, y_train)
print(rf.score(X_train, y_train))
print(rf.score(X_test, y_test))

0.9658024515673175
0.7498928447642537


In [316]:
gs = GridSearchCV(GBR(),
                  param_grid={'n_estimators': [100], 'max_depth':range(5,21,2), 'max_features':range(1,13,2) },
                  scoring=scoring, refit='r2', return_train_score=True, n_jobs=-1)
gs.fit(X_train, y_train)
gs.best_params_

{'max_depth': 5, 'max_features': 11, 'n_estimators': 100}

In [323]:
gbr = GBR(n_estimators=100, max_depth=5, max_features=11, random_state=123)
gbr.fit(X_train, y_train)
print(gbr.score(X_train, y_train))
print(gbr.score(X_test, y_test))

0.9735604236158247
0.8576112764160808


И случайный лес, и градиентный бустинг уже дают гораздо лучший результат!
$$R^2 = 85.76\%$$

Добавим функции от признаков, произведение з/п и безработицы, доходов и безработицы, а также отношение зарплаты и доходов.

In [325]:
current_features_count=X.shape[1]
for i in range(current_features_count):
    for fn in functions:
        X =np.concatenate((X,fn(X[:,i][:,None])), axis=1)
#         print(i, X.shape)

In [326]:
X=np.concatenate((X,(X[:,0]*X[:,1])[:,None]), axis=1)
X=np.concatenate((X,(X[:,6]*X[:,1])[:,None]), axis=1)
X=np.concatenate((X,(X[:,0]/X[:,6])[:,None]), axis=1)


In [327]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    shuffle=True,
                                                    random_state=42)

In [329]:
gs = GridSearchCV(GBR(),
                  param_grid={'n_estimators': [200], 'max_depth':range(4,21,2), 'max_features':range(6,50,2) },
                  scoring=scoring, refit='r2', return_train_score=True, n_jobs=-1)
gs.fit(X_train, y_train)
gs.best_params_

{'max_depth': 4, 'max_features': 6, 'n_estimators': 200}

In [331]:
gbr = GBR(n_estimators=300, max_depth=4, max_features=6, random_state=0)
gbr.fit(X_train, y_train)
print(gbr.score(X_train, y_train))
print(gbr.score(X_test, y_test))

0.9911138013559764
0.8719116002038028


In [332]:
gs = GridSearchCV(RandomForestRegressor(),
                  param_grid={'n_estimators': range(100, 200, 100), 
                              'max_depth':range(5,21,2), 
                              'max_features':range(5,55,3)},
                  scoring=scoring, refit='r2', return_train_score=True, n_jobs=-1)
gs.fit(X_train, y_train)
gs.best_params_

{'max_depth': 19, 'max_features': 14, 'n_estimators': 100}

In [333]:
rf = RandomForestRegressor(n_estimators=1000,max_depth=13, max_features=14, random_state=123)
rf.fit(X_train, y_train)
print(rf.score(X_train, y_train))
print(rf.score(X_test, y_test))

0.9662485532658681
0.74048645198644


С новыми признаками качетсво модели повысилось до $87\%$.

Еще одни признак, который можно добавить - уровень образования. Я нашла данные по численности студентов в ВУЗах на 10000 населения за 2005-2016 годы.

In [339]:
edu_url='http://nonerg-econ.ru/rdata/20/'
edu_page=requests.get(edu_url)

soup = BeautifulSoup(edu_page.text, 'html.parser')
soup.table.find_all('th')
edu_05_table = np.array([x.text for x in (soup.table.find_all('td'))]).reshape(-1,6)
edu_05_df = pd.DataFrame({2005:edu_05_table[:,2]}, index=edu_05_table[:,1])
edu_05_df.head()

,2005
г.Москва,"100,00"
г.Санкт-Петербург,"81,46"
Томская область,"64,40"
Тюменская область,"43,60"
Орловская область,"46,74"


Здесь тоже отсутствует пробел в названиях городов, исправим это. И объединим данные за все годы в одну таблицу.
Так же, как и для прошлых признаков, добавим средний уровень образования по региону. И для 2000-2004 годов, информации о которых не нашлось, заполним пустоты этим средним значением.

In [342]:
addresses = list(range(20,30))+[77,86]
edu_dfs=[]
for i, address in enumerate(addresses):
    year = 2005+i
    edu_url=f'http://nonerg-econ.ru/rdata/{address}/'
    edu_page=requests.get(edu_url)
    soup = BeautifulSoup(edu_page.text, 'html.parser')
    edu_table = np.array([x.text for x in (soup.table.find_all('td'))]).reshape(-1,6)
    edu_table[np.where(edu_table=='г.Москва')] = 'г. Москва'
    edu_table[np.where(edu_table=='г.Санкт-Петербург')] = 'г. Санкт-Петербург'
    edu_dfs.append(pd.DataFrame({year:np.array([x.replace(',','.') for x in edu_table[:,2]]).astype(float)}, 
                                index=edu_table[:,1]))

edu=edu_dfs[0]
for df in edu_dfs[1:]:
    edu = edu.join(df)

In [343]:
edu_df=edu.unstack().to_frame().sort_index(level=1).rename(columns={0:'education'})
region_mean_edu = edu.agg(['mean'], axis=1)

In [344]:
extra_years = set([x[0] for x in full_table.index])-set(edu.columns)
for year in extra_years:
    edu[year]=region_mean_edu['mean']

In [345]:
edu_df=edu.unstack().to_frame().sort_index(level=1).rename(columns={0:'education'})
region_mean_edu = pd.DataFrame({idx:region_mean_edu['mean'] for idx in edu.columns}, index=edu.index)
region_mean_edu = region_mean_edu.unstack().to_frame().sort_index(level=1).rename(columns={0:'region_mean_education'})

In [346]:
full_table=full_table.join(region_mean_edu).join(edu_df).dropna()

Финальный вид таблицы признаков:

In [357]:
full_table.head()

,,criminality,salary,unemployment,year_mean_unemp,region_mean_unemp,income,employment,unemployment_count,year_mean_income,region_mean_income,year_mean_emp,region_mean_emp,region_mean_education,education
2000,Алтайский край,2143.0,1364.8,11.7,12.098925,8.525,148.973,55.4,148.973,251.308753,104.7034,57.713978,58.98,36.4225,36.4225
2001,Алтайский край,2093.0,1870.4,10.0,10.481720,8.525,123.691,55.0,123.691,209.820022,104.7034,57.694624,58.98,36.4225,36.4225
2002,Алтайский край,2057.0,2567.6,8.3,9.337634,8.525,106.076,58.1,106.076,186.075376,104.7034,58.972043,58.98,36.4225,36.4225
2003,Алтайский край,2434.0,3147.5,12.3,9.872043,8.525,155.305,54.9,155.305,193.757344,104.7034,58.497849,58.98,36.4225,36.4225
2004,Алтайский край,2315.0,3955.3,9.1,9.347312,8.525,117.596,59.0,117.596,184.970333,104.7034,59.308602,58.98,36.4225,36.4225


In [349]:
X = np.zeros((full_table.shape[0],full_table.shape[1]-1))
for i, col_name in enumerate(full_table.columns[1:]):
    print(col_name)
    X[:,i] = full_table[col_name]
y  = (np.array(full_table['criminality']))

salary
unemployment
year_mean_unemp
region_mean_unemp
income
employment
unemployment_count
year_mean_income
region_mean_income
year_mean_emp
region_mean_emp
region_mean_education
education


In [1252]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    shuffle=True,
                                                    random_state=42)

In [352]:
gs = GridSearchCV(RandomForestRegressor(),
                  param_grid={'n_estimators': range(100, 200, 100), 
                              'max_depth':range(5,21,2), 
                              'max_features':range(5,55,3)},
                  scoring=scoring, refit='r2', return_train_score=True, n_jobs=-1)
gs.fit(X_train, y_train)
gs.best_params_

{'max_depth': 13, 'max_features': 11, 'n_estimators': 100}

In [364]:
rf = RandomForestRegressor(n_estimators=100,max_depth=13, max_features=11, random_state=1)
rf.fit(X_train, y_train)
print(rf.score(X_train, y_train))
print(rf.score(X_test, y_test))

0.9647734829512301
0.7480396548225972


In [358]:
gs = GridSearchCV(GBR(),
                  param_grid={'n_estimators': [200], 'max_depth':range(4,21,2), 'max_features':range(6,50,2) },
                  scoring=scoring, refit='r2', return_train_score=True, n_jobs=-1)
gs.fit(X_train, y_train)
gs.best_params_

{'max_depth': 6, 'max_features': 8, 'n_estimators': 200}

In [363]:
gbr = GBR(n_estimators=200, max_depth=6, max_features=8, random_state=0)
gbr.fit(X_train, y_train)
print(gbr.score(X_train, y_train))
print(gbr.score(X_test, y_test))

0.999340990987435
0.8555147834553122


In [365]:
current_features_count=X.shape[1]
for i in range(current_features_count):
    for fn in functions:
        X =np.concatenate((X,fn(X[:,i][:,None])), axis=1)
X=np.concatenate((X,(X[:,0]*X[:,1])[:,None]), axis=1)
X=np.concatenate((X,(X[:,6]*X[:,1])[:,None]), axis=1)
X=np.concatenate((X,(X[:,0]/X[:,6])[:,None]), axis=1)

In [366]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    shuffle=True,
                                                    random_state=42)

In [373]:
rf = RandomForestRegressor(n_estimators=200,max_depth=13, max_features=11, random_state=123)
rf.fit(X_train, y_train)
print(rf.score(X_train, y_train))
print(rf.score(X_test, y_test))

0.9695557466769648
0.8232288804658388


In [384]:
gbr = GBR(n_estimators=400, max_depth=6, max_features=8, random_state=0)
gbr.fit(X_train, y_train)
print(gbr.score(X_train, y_train))
print(gbr.score(X_test, y_test))

0.9999908519475444
0.8900413532190986


Признак уровня образования сам по себе не улучшил качество регрессии, но в совокупности со своими "производными" (обратное, корень, логарифм), позволил достичь $89\%$ качества на тестовой выборке.

На самом деле есть еще множество признаков, которые можно было бы добавить: уровень бедности (доля населения, имеющего доход ниже некоторого порога,  как https://www.gks.ru/free_doc/new_site/population/bednost/tabl/tab-bed2-7.htm), степень расслоения общества (разброс степени обеспеченности населения), плотность населения в регионе, доступность оружия, уровень коррупции и т.д.). Однако доступ к таким данным за большой период времени(10 лет и более) ограничен. Можно найти некоторую статистику за отдельные годы, но добавления этих данных в выборку либо приведет к существунному уменьшению количества объектов в выборке (если рассматривать только объекты, для которых все известно), либо к большому количеству пропущенных значений.
Поэтому я остановлясь на текущей модели градиентного бустинга, предсказывающей уровень преступности по безработице, занятости, средней зарплате и доходам, а также уровню образования с $R^2=89\%$.